In [1]:
import pandas as pd
import re
from nltk import FreqDist
from tqdm import tqdm
tqdm.pandas() # for use progress_apply
from konlpy.tag import Okt, Kkma, Hannanum, Komoran

In [2]:
okt = Okt()
kkma = Kkma()
komoran = Komoran()
han = Hannanum()

In [3]:
df1 = pd.read_csv("20220111-20220119_naver_finance_news.csv",encoding="utf-8-sig")
df2 = pd.read_csv("20220103-20220110_naver_finance_news.csv",encoding="utf-8-sig")
df3 = pd.read_csv("20220120-20220128_naver_finance_news.csv",encoding="utf-8-sig")
df4 = pd.read_csv("20220204-20220207_naver_finance_news.csv",encoding="utf-8-sig")
df = pd.concat([df1, df2, df3, df4], axis=0)
df.reset_index(drop=True, inplace=True)
new_df = df.copy()
new_df

,제목,시간,언론사,본문
0,옥석 가려야 한다는 골드만의 때늦은 경고…3대 지수 하락 출발 [뉴욕증시 나우],2022-01-11 23:59,한국경제TV,여기는 미국 동부시간 11일 오전 9시 31분입니다. 어제 장 막판 나스닥이 극적으...
1,"모건스탠리 ""테슬라, 전기차 시장의 리더""…목표주가 1300달러 [강영연의 뉴욕오프닝]",2022-01-11 23:21,한국경제,이 기사는 국내 최대 해외 투자정보 플랫폼 “한경 글로벌마켓”에 게재된 기사입니다....
2,"앨버트슨, 4분기 순이익 4억2450만달러...전년비 두 배↑",2022-01-11 22:43,한국경제TV,미국의 식료품 소매업체 앨버트슨 컴퍼니(Albertsons Companies)가 예...
3,"플래닛피트니스, 선샤인피트니스 8억달러에 인수",2022-01-11 22:22,한국경제TV,미국의 피트니스 프랜차이즈 플래닛피트니스는 선샤인피트니스를 현금과 주식으로 8억 달...
4,"4년 전에도 금리 인상 '속도전' 치른 연준... 월가 ""올해도 네 번 올릴 듯""",2022-01-11 22:01,한국일보,"글로벌 은행 ""3, 6, 9, 12월 올릴 듯"" 2018년에도 네 차례 인상... ..."
...,...,...,...,...
22845,"“美주식거래 이젠 낮에 하세요”… 삼성증권, 첫 주간거래 서비스",2022-02-07 03:02,동아일보,"美 대체거래소 ‘블루오션’ 독점제휴, 오늘부터 오전10시~오후 5시반모든 종목 정규..."
22846,증권사 역대급 실적… ‘1조 클럽’ 1곳서 5곳으로,2022-02-07 03:02,동아일보,작년 증시 호황에 영업익 크게 늘어일부 현금배당 등 적극적 주주 환원지난해 증시 호...
22847,[코스닥 주간메모] 2월 7일(월) ~ 2월 11일(금),2022-02-07 01:43,한국경제,[2월 7일(월)]◇ 신규상장 = 아셈스(136410)◇ 공모주청약 = 인카금융서비...
22848,[유가증권 주간메모] 2월 7일(월) ~ 2월 11일(금),2022-02-07 01:42,한국경제,[2월 7일(월)]◇ 변경상장 = 대한전선(자본감소)◇ 보통주추가상장 = 두산중공업...


In [4]:
title = new_df['제목'].progress_apply(lambda x: okt.pos(x,join=True)) 
content = new_df['본문'].progress_apply(lambda x: str(x))
content = content.progress_apply(lambda x: okt.pos(x,join=True))
words = content + title

100%|██████████| 22850/22850 [07:22<00:00, 51.67it/s] 


## 1. Title

In [6]:
noun = re.compile('.*/Noun$') # 명사
verb = re.compile('.*/Verb$') # 동사
adj = re.compile('.*/Adjective$') # 형용사
foreign = re.compile('.*/Foreign$') # 외국어, 한자 및 기타기호
alpha = re.compile('.*/Alpha$') #알파벳

wordlist = []
for list in title:
    for word in list:
        if noun.match(word) or verb.match(word) or adj.match(word) or foreign.match(word) or alpha.match(word):
            wordlist.append(word)

In [7]:
freqdist = FreqDist(wordlist)
title_wordbag = {'word':[],'frequency':[],'pos':[]}
for element in freqdist:
    title_wordbag['word'].append(element)
    title_wordbag['frequency'].append(freqdist[element])

In [8]:
i = 0
for pair in title_wordbag['word']:
    word, pos = pair.split('/')
    title_wordbag['pos'].append(pos)
    title_wordbag['word'][i] = word
    i += 1

In [9]:
data = pd.DataFrame(title_wordbag)
data['sentiment'] = ''
data['stopword'] = ''
data['headword'] = ''
data['attribute'] = ''
data = data[data["frequency"]>=1]
data.to_csv("title_dictionary.csv",encoding='utf-8-sig',index=False)

In [10]:
display(data)

,word,frequency,pos,sentiment,stopword,headword,attribute
0,코스피,2533,Noun,,,,
1,주,2096,Noun,,,,
2,LG,1872,Alpha,,,,
3,증권,1548,Noun,,,,
4,마감,1506,Noun,,,,
...,...,...,...,...,...,...,...
13496,새도,1,Verb,,,,
13497,좋아도,1,Adjective,,,,
13498,없으면,1,Adjective,,,,
13499,염불,1,Noun,,,,


## 2. Title + Content

In [28]:
noun = re.compile('.*/Noun$') # 명사
verb = re.compile('.*/Verb$') # 동사
adj = re.compile('.*/Adjective$') # 형용사
foreign = re.compile('.*/Foreign$') # 외국어, 한자 및 기타기호
alpha = re.compile('.*/Alpha$') #알파벳

wordlist = []
for list in words:
    for word in list:
        if noun.match(word) or verb.match(word) or adj.match(word) or foreign.match(word) or alpha.match(word):
            wordlist.append(word)

In [29]:
freqdist = FreqDist(wordlist)
wordbag = {'word':[],'frequency':[],'pos':[]}
for element in freqdist:
    wordbag['word'].append(element)
    wordbag['frequency'].append(freqdist[element])

In [30]:
i = 0
for pair in wordbag['word']:
    word, pos = pair.split('/')
    wordbag['pos'].append(pos)
    wordbag['word'][i] = word
    i += 1

In [56]:
data = pd.DataFrame(wordbag)
data['sentiment'] = ''
data['stopword'] = ''
data['headword'] = ''
data['attribute'] = ''
data = data[data["frequency"]>=1]
data.to_csv("dictionary.csv",encoding='utf-8-sig',index=False)

In [57]:
display(data)

,word,frequency,pos,sentiment,stopword,headword,attribute
0,했다,76725,Verb,,,,
1,것,48044,Noun,,,,
2,있다,40908,Adjective,,,,
3,등,38211,Noun,,,,
4,증권,32602,Noun,,,,
...,...,...,...,...,...,...,...
67388,반모든,1,Foreign,,,,
67389,못자며,1,Verb,,,,
67390,강모,1,Noun,,,,
67391,설치던,1,Adjective,,,,
